# Face Metrics Experiments

In [1]:
import sys
sys.path.append("..")

from newMesh import FaceTetrahedronMesh
from PolyllaFace import PolyllaFace
import time
import os
import pathlib

polylla_inradoius = []
polylla_circumradoius = []
polylla_area = []

inradious = 'r'
circumradious = 'R'
area = 'a'

n_experiments = 5

In [2]:
def run_polylla(n, distribution, flag):
    filename = "../data/"+ str(n) + distribution +".1"
    node_file = filename + ".node"
    ele_file = filename + ".ele"
    face_file = filename + ".face"

    t0 = time.time()
    mesh= FaceTetrahedronMesh(node_file, face_file, ele_file)
    polyllaFace_mesh = PolyllaFace(mesh,flag)
    tf = time.time()

    dtM = tf - t0 

    print('Tiempo:',dtM,'segs')
    edge_ratio = polyllaFace_mesh.edge_ratio()
    tetra_per_poly = polyllaFace_mesh.tetra_per_poly()
    faces_per_poly = polyllaFace_mesh.faces_per_poly()
    polyhedron_volume = polyllaFace_mesh.polyhedron_volume()
    polyhedron_area_ratio = polyllaFace_mesh.polyhedron_area()
    # print(polyllaFace_mesh.mesh.face_list[4],polyllaFace_mesh.mesh.face_list[12])
    print('\n Stats polylla:')
    polyllaFace_mesh.get_info()
    filename = 'polylla_'+distribution+str(n)+flag
    polyllaFace_mesh.writePolygonFile(filename)
    # print(polyllaFace_mesh.mesh.face_list[4],polyllaFace_mesh.mesh.face_list[12])
    #polyllaFace_mesh.printOFF_each_poly('../logs/polys'+flag+filename)
    os.system("./kernel "+ filename +".txt")
    polyhedrom_volume_ratio = polyllaFace_mesh.volume_ratio(filename+'_kernel_volumes')
    print('Mean Edge Ratio:', edge_ratio[0])
    print('Minimum Edge Ratio:', edge_ratio[1])
    print('Maximum Edge Ratio:', edge_ratio[2])
    print('Mean Tetrahedron per Polyhedron:', tetra_per_poly[0])
    print('Minimum Tetrahedron per Polyhedron:', tetra_per_poly[1])
    print('Maximum Tetrahedron per Polyhedron:', tetra_per_poly[2])
    print('Mean Faces per Polyhedron:', faces_per_poly[0])
    print('Minimum Faces per Polyhedron:', faces_per_poly[1])
    print('Maximum Faces per Polyhedron:', faces_per_poly[2])
    print('Mean Volume Ratio:', polyhedrom_volume_ratio[0])
    print('Minimum Volume Ratio:', polyhedrom_volume_ratio[1])
    print('Maximum Volume Ratio:', polyhedrom_volume_ratio[2])
    print('Polyhedrons with kernel:', round(polyhedrom_volume_ratio[3],3), '%')
    print('Polyhedrons are convex:', polyhedrom_volume_ratio[4], '%')
    print('Mean Area Ratio :', polyhedron_area_ratio[0])
    print('Minimum Area Ratio :', polyhedron_area_ratio[1])
    print('Maximum Area Ratio :', polyhedron_area_ratio[2])
    


    print()
    return polyllaFace_mesh

## Random Distribution

In [3]:
distribution = 'random'

#### n = 100

In [4]:
n = 20

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
# polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
# polylla_mesh_area = run_polylla(n,distribution,area)

# polylla_inradoius.append(polylla_mesh_inradious)
# polylla_circumradoius .append(polylla_circumradoius)
# polylla_area.append(polylla_area)



Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.004026174545288086 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 21
Number of barrier faces: 6
Number of polyhedra with barrier faces: 4
Number of polyhedrons that are tetrahedrons: 4
writing OFF files: polylla_random20r
Input: polylla_random20r.txt
21 polygons
Mean Edge Ratio: 0.10140348445822693
Minimum Edge Ratio: 0.003162672150869759
Maximum Edge Ratio: 0.24031293944970084
Mean Tetrahedron per Polyhedron: 2.619047619047619
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 7
Mean Faces per Polyhedron: 7.238095238095238
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 16
Mean Volume Ratio: 0.5938278013886996
Minimum Volume Ratio: 0.07969051173525211
Maximum Volume Ratio: 1.0000050698031357
Polyhedrons with kernel: 90.476 %
Mean Area Ratio : 0.9682813845206638
Minimum Area Ratio : 0.8306407937537869
Maximum Area 

#### n = 500

In [5]:
n = 500

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.13206195831298828 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 726
Number of barrier faces: 209
Number of polyhedra with barrier faces: 145
Number of polyhedrons that are tetrahedrons: 136
Mean Edge Ratio: 0.098309648651974
Minimum Edge Ratio: 2.4934446865405707e-05
Maximum Edge Ratio: 0.5096617549864849
Mean Tetrahedron per Polyhedron: 3.275482093663912
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 18
Mean Faces per Polyhedron: 8.550964187327823
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 38
Mean Volume per Polyhedron: 0.0013774104683195593
Minimum Volume per Polyhedron: 3.4147111137719004e-06
Maximum Volume per Polyhedron: 0.008886345135158378
Mean Area Ratio per Polyhedron: 0.9948186574211939
Minimum Area Ratio per Polyhedron: 0.6401515539136479
Maximum Area Ratio per Polyhedron: 1.2680399815162298

R

#### n = 1000

In [6]:
n = 1000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.3451263904571533 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 1456
Number of barrier faces: 431
Number of polyhedra with barrier faces: 279
Number of polyhedrons that are tetrahedrons: 253
Mean Edge Ratio: 0.0956392082410924
Minimum Edge Ratio: 1.2083492353995125e-06
Maximum Edge Ratio: 0.4778453141461106
Mean Tetrahedron per Polyhedron: 3.3839285714285716
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 19
Mean Faces per Polyhedron: 8.767857142857142
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 40
Mean Volume per Polyhedron: 0.0006868131868131869
Minimum Volume per Polyhedron: 4.366002972154357e-06
Maximum Volume per Polyhedron: 0.007685410952970693
Mean Area Ratio per Polyhedron: 0.994143962760562
Minimum Area Ratio per Polyhedron: 0.5439572877932475
Maximum Area Ratio per Polyhedron: 1.212692741071989

Re

##### n = 5000

In [7]:
n = 5000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 1.7490286827087402 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 6990
Number of barrier faces: 1975
Number of polyhedra with barrier faces: 1299
Number of polyhedrons that are tetrahedrons: 1596
Mean Edge Ratio: 0.07287930039138193
Minimum Edge Ratio: 5.7472427817186655e-09
Maximum Edge Ratio: 0.452153945188356
Mean Tetrahedron per Polyhedron: 3.390844062947067
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 21
Mean Faces per Polyhedron: 8.781688125894135
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 44
Mean Volume per Polyhedron: 0.0001430615164520744
Minimum Volume per Polyhedron: 7.611697301975154e-08
Maximum Volume per Polyhedron: 0.0024592215761638572
Mean Area Ratio per Polyhedron: 0.9975181766312968
Minimum Area Ratio per Polyhedron: 0.5323515653917708
Maximum Area Ratio per Polyhedron: 1.580296119501123

#### n = 10000

In [8]:
n = 10000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius.append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 3.8231544494628906 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 14440
Number of barrier faces: 3871
Number of polyhedra with barrier faces: 2619
Number of polyhedrons that are tetrahedrons: 3537
Mean Edge Ratio: 0.07087612569818641
Minimum Edge Ratio: 2.8162128837440787e-10
Maximum Edge Ratio: 0.5256758465615928
Mean Tetrahedron per Polyhedron: 3.280747922437673
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 25
Mean Faces per Polyhedron: 8.561495844875346
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 52
Mean Volume per Polyhedron: 6.924906131610416e-05
Minimum Volume per Polyhedron: 5.819361916070612e-08
Maximum Volume per Polyhedron: 0.002173204175544548
Mean Area Ratio per Polyhedron: 0.9979301689636738
Minimum Area Ratio per Polyhedron: 0.44071934397684137
Maximum Area Ratio per Polyhedron: 1.4909983675324

In [9]:
n = 50000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius.append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 16.304452419281006 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 75543
Number of barrier faces: 18245
Number of polyhedra with barrier faces: 12202
Number of polyhedrons that are tetrahedrons: 22872
Mean Edge Ratio: 0.06708836060723614
Minimum Edge Ratio: 3.718387291220964e-11
Maximum Edge Ratio: 0.5866525596361951
Mean Tetrahedron per Polyhedron: 3.0505407516249026
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 35
Mean Faces per Polyhedron: 8.101081503249805
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 72
Mean Volume per Polyhedron: 1.3230441727749802e-05
Minimum Volume per Polyhedron: 1.8420617560436782e-09
Maximum Volume per Polyhedron: 0.001151188248176862
Mean Area Ratio per Polyhedron: 0.9985598400615772
Minimum Area Ratio per Polyhedron: 0.3135161904626988
Maximum Area Ratio per Polyhedron: 2.308721119

## Poisson Distribution

In [6]:
distribution = 'poisson'

#### n = 100

In [7]:
n = 100

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.12005734443664551 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 798
Number of barrier faces: 200
Number of polyhedra with barrier faces: 153
Number of polyhedrons that are tetrahedrons: 147
Mean Edge Ratio: 0.25296107506391546
Minimum Edge Ratio: 0.016140368797698593
Maximum Edge Ratio: 0.5865062280322514
Mean Tetrahedron per Polyhedron: 3.112781954887218
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.225563909774436
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26
Mean Volume per Polyhedron: 0.0012531328320802004
Minimum Volume per Polyhedron: 1.8427115244969453e-05
Maximum Volume per Polyhedron: 0.008676829494942443
Mean Area Ratio per Polyhedron: 0.9986131580541983
Minimum Area Ratio per Polyhedron: 0.8338629587256614
Maximum Area Ratio per Polyhedron: 1.1376299635430083

R

#### n = 500

In [18]:
n = 500

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.12086248397827148 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 774
Number of barrier faces: 216
Number of polyhedra with barrier faces: 157
Number of polyhedrons that are tetrahedrons: 127
Mean Edge Ratio: 0.2549823219844767
Minimum Edge Ratio: 7.647544771746064e-05
Maximum Edge Ratio: 0.5619177012352183
Mean Tetrahedron per Polyhedron: 3.1847545219638245
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.369509043927648
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28
Mean Volume per Polyhedron: 0.0012919896640826874
Minimum Volume per Polyhedron: 6.146988497465139e-06
Maximum Volume per Polyhedron: 0.00832455340496012
Mean Area Ratio per Polyhedron: 0.9972218282098875
Minimum Area Ratio per Polyhedron: 0.8746095617648036
Maximum Area Ratio per Polyhedron: 1.2702215279972109

Re

#### n = 1000

In [19]:
n = 1000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.11839771270751953 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 751
Number of barrier faces: 196
Number of polyhedra with barrier faces: 132
Number of polyhedrons that are tetrahedrons: 114
Mean Edge Ratio: 0.2541589933016368
Minimum Edge Ratio: 0.01204919504651487
Maximum Edge Ratio: 0.7742445817199215
Mean Tetrahedron per Polyhedron: 3.2343541944074565
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.468708388814914
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28
Mean Volume per Polyhedron: 0.0013315579227696406
Minimum Volume per Polyhedron: 3.704690155034038e-05
Maximum Volume per Polyhedron: 0.007974913690740743
Mean Area Ratio per Polyhedron: 0.9984779876782569
Minimum Area Ratio per Polyhedron: 0.8895974957712197
Maximum Area Ratio per Polyhedron: 1.2306135276199643

Rea

##### n = 5000

In [20]:
n = 5000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.11505866050720215 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 782
Number of barrier faces: 209
Number of polyhedra with barrier faces: 150
Number of polyhedrons that are tetrahedrons: 136
Mean Edge Ratio: 0.24639963317288865
Minimum Edge Ratio: 0.0057801751149192775
Maximum Edge Ratio: 0.6129189204628973
Mean Tetrahedron per Polyhedron: 3.0920716112531967
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.184143222506394
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28
Mean Volume per Polyhedron: 0.0012787723785166241
Minimum Volume per Polyhedron: 3.2913796634441974e-05
Maximum Volume per Polyhedron: 0.01011146437869377
Mean Area Ratio per Polyhedron: 0.9982816527293884
Minimum Area Ratio per Polyhedron: 0.818072886278701
Maximum Area Ratio per Polyhedron: 1.1331243224334957

R

#### n = 10000

In [21]:
n = 10000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.11952328681945801 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 746
Number of barrier faces: 185
Number of polyhedra with barrier faces: 136
Number of polyhedrons that are tetrahedrons: 125
Mean Edge Ratio: 0.2559774259301135
Minimum Edge Ratio: 0.009591936163852031
Maximum Edge Ratio: 0.5340259938083121
Mean Tetrahedron per Polyhedron: 3.2319034852546915
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 13
Mean Faces per Polyhedron: 8.463806970509383
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 28
Mean Volume per Polyhedron: 0.0013404825737265416
Minimum Volume per Polyhedron: 5.0688078083113393e-05
Maximum Volume per Polyhedron: 0.008112819803751984
Mean Area Ratio per Polyhedron: 0.9980448705688049
Minimum Area Ratio per Polyhedron: 0.8469470104266235
Maximum Area Ratio per Polyhedron: 1.1459204679341055

R

## Semiuniform Distribution

In [22]:
distribution = 'semiuniform'

#### n = 100

In [23]:
n = 100

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.1122288703918457 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26
Mean Volume per Polyhedron: 0.0014064697609001407
Minimum Volume per Polyhedron: 8.862595790489203e-05
Maximum Volume per Polyhedron: 0.005885528603360098
Mean Area Ratio per Polyhedron: 0.9950788013925883
Minimum Area Ratio per Polyhedron: 0.8847900721468468
Maximum Area Ratio per Polyhedron: 1.1782994373449838

Rea

#### n = 500

In [24]:
n = 500

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.11822247505187988 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26
Mean Volume per Polyhedron: 0.0014064697609001407
Minimum Volume per Polyhedron: 8.862595790489203e-05
Maximum Volume per Polyhedron: 0.005885528603360098
Mean Area Ratio per Polyhedron: 0.9950788013925883
Minimum Area Ratio per Polyhedron: 0.8847900721468468
Maximum Area Ratio per Polyhedron: 1.1782994373449838

Re

#### n = 1000

In [25]:
n = 1000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.09732365608215332 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26
Mean Volume per Polyhedron: 0.0014064697609001407
Minimum Volume per Polyhedron: 8.862595790489203e-05
Maximum Volume per Polyhedron: 0.005885528603360098
Mean Area Ratio per Polyhedron: 0.9950788013925883
Minimum Area Ratio per Polyhedron: 0.8847900721468468
Maximum Area Ratio per Polyhedron: 1.1782994373449838

Re

##### n = 5000

In [26]:
n = 5000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.09233856201171875 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26
Mean Volume per Polyhedron: 0.0014064697609001407
Minimum Volume per Polyhedron: 8.862595790489203e-05
Maximum Volume per Polyhedron: 0.005885528603360098
Mean Area Ratio per Polyhedron: 0.9950788013925883
Minimum Area Ratio per Polyhedron: 0.8847900721468468
Maximum Area Ratio per Polyhedron: 1.1782994373449838

Re

#### n = 10000

In [27]:
n = 10000

polylla_mesh_inradious = run_polylla(n,distribution,inradious)
polylla_mesh_circumradious = run_polylla(n,distribution,circumradious)
polylla_mesh_area = run_polylla(n,distribution,area)

polylla_inradoius.append(polylla_mesh_inradious)
polylla_circumradoius .append(polylla_circumradoius)
polylla_area.append(polylla_area)

Reading vertex file
Reading face file
Processing edges
Reading tetra file
Processesing faces with tetrahedorns
Tiempo: 0.09666752815246582 segs

 Stats polylla:
PolyllaFace info:
Number of polyhedrons: 711
Number of barrier faces: 198
Number of polyhedra with barrier faces: 137
Number of polyhedrons that are tetrahedrons: 129
Mean Edge Ratio: 0.33616843588457035
Minimum Edge Ratio: 0.09759307171854792
Maximum Edge Ratio: 0.6564894374861545
Mean Tetrahedron per Polyhedron: 3.1322081575246132
Minimum Tetrahedron per Polyhedron: 1
Maximum Tetrahedron per Polyhedron: 12
Mean Faces per Polyhedron: 8.264416315049226
Minimum Faces per Polyhedron: 4
Maximum Faces per Polyhedron: 26
Mean Volume per Polyhedron: 0.0014064697609001407
Minimum Volume per Polyhedron: 8.862595790489203e-05
Maximum Volume per Polyhedron: 0.005885528603360098
Mean Area Ratio per Polyhedron: 0.9950788013925883
Minimum Area Ratio per Polyhedron: 0.8847900721468468
Maximum Area Ratio per Polyhedron: 1.1782994373449838

Re